ЗАДАНИЕ 1

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

In [2]:
reg_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-maksim-daragan-mxh-056d6/shared/problem1-reg_data.csv')
auth_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-maksim-daragan-mxh-056d6/shared/problem1-auth_data.csv')

In [3]:
reg_data[['reg_ts', 'uid']] = reg_data['reg_ts;uid'].str.split(';', expand=True)
auth_data[['auth_ts', 'uid']] = auth_data['auth_ts;uid'].str.split(';', expand=True)
#создаём новый столбец для удобного отображения данных

In [4]:
reg_data.drop(columns=['reg_ts;uid'], inplace=True)
auth_data.drop(columns=['auth_ts;uid'], inplace=True)
#удаляем лишний столбец

In [5]:
reg_data['reg_time'] = pd.to_datetime(reg_data['reg_ts'], unit='s', utc=True).dt.date
auth_data['auth_time'] = pd.to_datetime(auth_data['auth_ts'], unit='s', utc=True).dt.date
#преобразуем количество секунд с начала эпохи Unix в дату, оставив только дни

In [6]:
reg_data['reg_time'] = pd.to_datetime(reg_data['reg_time'])
auth_data['auth_time'] = pd.to_datetime(auth_data['auth_time'])
#переводим даты в формат datetime

In [7]:
union_df = pd.merge(reg_data, auth_data, on='uid')
#объединяем датафреймы в один

In [8]:
union_df.drop(columns=['reg_ts', 'auth_ts'], inplace=True)
#убираем лишние колонки

In [9]:
union_df['reg_time'] = pd.to_datetime(union_df['reg_time'])
union_df['auth_time'] = pd.to_datetime(union_df['auth_time'])
#переводим даты в формат datetime

In [10]:
def daily_retention(auth_data, reg_data):
     union_df['days_since_registration'] = (union_df['auth_time'] - union_df['reg_time']).dt.days
     retention = union_df.groupby('days_since_registration')['uid'].nunique().reset_index()   
     retention.columns = ['days_since_registration', 'retained_users']
        
     return retention   
#прописываем функцию для подсчёта retention игроков (по дням от даты регистрации игрока)

In [11]:
retention_result = daily_retention(auth_data, reg_data)
print(retention_result)
#вычисляем retention

      days_since_registration  retained_users
0                           0         1000000
1                           1           20071
2                           2           40997
3                           3           46338
4                           4           52258
...                       ...             ...
5911                     7716               1
5912                     7720               1
5913                     7721               1
5914                     7727               1
5915                     7729               1

[5916 rows x 2 columns]


In [53]:
def calculate_cohort_retention(reg_data, auth_data):         
  
    merged_data = pd.merge(reg_data, auth_data, on='uid', how='left')
    #Объединяем данные о регистрации и авторизации
    
    merged_data['days_since_registration'] = (merged_data['auth_time'] - merged_data['reg_time']).dt.days
    #Вычисляем количество дней с момента регистрации
    
    merged_data = merged_data[merged_data['days_since_registration'] >= 0]
    #Фильтруем только те записи, где авторизация произошла после регистрации
    
    cohort_data = merged_data.groupby(['reg_time', 'days_since_registration'])['uid'].nunique().reset_index()
    cohort_data.columns = ['reg_date', 'days_since_registration', 'retention_count']
    #Группируем по дате регистрации и количеству дней с момента регистрации
    
    cohort_pivot = cohort_data.pivot(index='reg_date', columns='days_since_registration', values='retention_count').fillna(0)
    #Создаем сводную таблицу для удобства анализа
    
    return cohort_pivot

In [54]:
cohort_results = calculate_cohort_retention(reg_data, auth_data)

print(cohort_results)
#Выводим результаты

days_since_registration    0     1     2     3     4     5     6     7     \
reg_date                                                                    
1998-11-18                  1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1999-07-22                  1.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   
2000-01-13                  1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2000-05-28                  1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2000-09-16                  1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...                         ...   ...   ...   ...   ...   ...   ...   ...   
2020-09-19               1634.0  30.0  58.0  77.0  57.0   0.0   0.0   0.0   
2020-09-20               1636.0  40.0  71.0  39.0   0.0   0.0   0.0   0.0   
2020-09-21               1638.0  31.0  49.0   0.0   0.0   0.0   0.0   0.0   
2020-09-22               1641.0  14.0   0.0   0.0   0.0   0.0   0.0   0.0   
2020-09-23               1048.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

ЗАДАНИЕ 2

Метрика ARPU (Average Revenue Per User)

In [12]:
ab_df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-maksim-daragan-mxh-056d6/final_project_da/Проект_1_Задание_2.csv')

In [13]:
ab_df[['user_id', 'revenue', 'testgroup']] = ab_df['user_id;revenue;testgroup'].str.split(';', expand=True)
#делаем датафрейм красивым и читаемым, разделяя столбцы

In [14]:
ab_df.drop(columns=['user_id;revenue;testgroup'], inplace=True)
#удаляем лишние столбцы

In [15]:
ab_df['revenue'] = ab_df['revenue'].astype(int)
#преобразуем значения столбца revenue в целочисленное

In [16]:
control = ab_df[ab_df['testgroup'] == 'a'] 
test = ab_df[ab_df['testgroup'] == 'b'] 
#делим датафрейм на тестовую и контрольную группу

In [17]:
control_arpu = control['revenue'].sum() / len(control)
test_arpu = test['revenue'].sum() / len(test)
#считаем ARPU для тестовой и контрольной группы

In [18]:
t_stat, p_value = stats.ttest_ind(control['revenue'], test['revenue'], equal_var=False)
#проводим t-тест для того, чтобы узнать, являются ли наши различия статистически значимыми

In [19]:
alpha = 0.05
#обозначаем уровень значимости

In [20]:
if p_value < alpha:
    difference = True
else:
    difference = False
#проверяем, является ли разница статистически значимой

In [21]:
if difference and test_arpu > control_arpu:
    best_group = 'test'
else:
    best_group = 'control'

print(f'Лучшая группа: {best_group}')
#выводим результаты для выявления лучшей группы на основании ARPU

Лучшая группа: control


Метрика ARPPU (Average Revenue Per Paying User)

In [22]:
pay_ab_df = ab_df[ab_df['revenue'] > 0]
#выделяем в датафрейме только платящих пользователей

In [23]:
pay_control = pay_ab_df[pay_ab_df['testgroup'] == 'a'] 
pay_test = pay_ab_df[pay_ab_df['testgroup'] == 'b'] 
#делим датафрейм на тестовую и контрольную группу

In [24]:
control_arppu = pay_control['revenue'].sum() / len(pay_control)
test_arppu = pay_test['revenue'].sum() / len(pay_test)
#считаем ARPPU для тестовой и контрольной группы

In [25]:
t_stat, p_value = stats.ttest_ind(pay_control['revenue'], pay_test['revenue'], equal_var=False)
#проводим t-тест для того, чтобы узнать, являются ли наши различия статистически значимыми

In [26]:
if p_value < alpha:
    difference_pay = True
else:
    difference_pay = False
#проверяем, является ли разница статистически значимой

In [27]:
if difference_pay and test_arppu > control_arppu:
    best_group_pay = 'test'
else:
    best_group_pay = 'control'

print(f'Лучшая группа: {best_group_pay}')
#выводим результаты для выявления лучшей группы на основании ARPPU

Лучшая группа: control


Метрика Conversion Rate (Конверсия)

In [28]:
control_pay_users = 1928
test_pay_users = 1805
#обозначаем количество платящих пользователей в тестовой и контрольной группах

In [29]:
control_total_users = 202103
test_total_users = 202667
#обозначаем общее количество пользователей в тестовой и контрольной группах

In [30]:
control_conversion = (control_pay_users / control_total_users) * 100
test_conversion = (test_pay_users / test_total_users) * 100
#считаем конверсию для тестовой и контрольной группы

In [31]:
pay = [control_pay_users, test_pay_users]
total = [control_total_users, test_total_users]
z_stat, p_value = proportions_ztest(pay, total)
#проводим z-тест для пропорций, чтобы проверить, есть ли статистически значимая разница в конверсии между группами

In [32]:
difference_conversion = p_value < alpha
#проверяем, является ли разница статистически значимой

In [33]:
if difference_conversion and test_conversion > control_conversion:
    best_group_conv = 'test'
else:
    best_group_conv = 'control'

print(f'Лучшая группа: {best_group_conv}')
#выводим результаты для выявления лучшей группы на основании Conversion Rate

Лучшая группа: control


Вывод: набор предложений в контрольной группе можно считать лучшим на основании метрик ARPU и Conversion Rate.

ЗАДАНИЕ 3

В игре Plants & Gardens каждый месяц проводятся тематические события, ограниченные по времени. В них игроки могут получить уникальные предметы для сада и персонажей, дополнительные монеты или бонусы. Для получения награды требуется пройти ряд уровней за определенное время. С помощью каких метрик можно оценить результаты последнего прошедшего события?

Основные метрики для оценки результатов прошедшего события:
    1. Общее количество пользователей, принявших участие в событии: метрика нужна для понимания интереса и привлекательности
        ивента.
    2. Процент завершения события: метрика позволяет оценить вовлечённость пользователей в ивент.
    3. Среднее время прохождения: метрика позволяет оценить, насколько хорошо сбалансированы уровни по времени и сложности.

Предположим, в другом событии мы усложнили механику событий так, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад. Изменится ли набор метрик оценки результата? Если да, то как?

При усложнении механики я бы оставил предыдущие метрики, так как они являются ключевыми при оценке результата, но при этом добавил к ним новые:
    1. Количество откатов: метрика позволяет оценить, насколько часто игроки совершают неудачные попытки.
    2. Процент неудачных попыток: метрика позволяет понять, насколько сложными стали уровни.
    3. Среднее время на каждый уровень: метрика позволяет понять, как усложнение механики влияет на общее время прохождения.